In [31]:
import tensorflow as tf
from tensorflow import keras

from keras.utils import load_img

import numpy as np
import os
import pickle

## 1. Instantiate the model

In [32]:
vgg_model = keras.applications.vgg16.VGG16(weights='imagenet')

## 2. Create a base model
We exclude the top part, referring to the classification layers. Doing so our model is capable of extracting features.

In [33]:
# Use the resolution that was chosen to take the photos of your objects
base_model = keras.applications.vgg16.VGG16(weights='imagenet',
                                            input_shape=(650,650,3),
                                            include_top=False)

In [34]:
for layers in base_model.layers[:]:
    layers.trainable = False

In [35]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 650, 650, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 650, 650, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 650, 650, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 325, 325, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 325, 325, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 325, 325, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 162, 162, 128)     0     

## 3. Load the data

In [36]:
X = []
y = []
# The model was only trained on the two most similar jerseys
classes = ["object3", "object4"]
base_path = './photos/'

for i, target in enumerate(classes):
    files = os.listdir(base_path+target)
    for file in files:
        path = base_path + target + "/" + file
        # load the image
        img = load_img(
            path=path,
            grayscale=False,
            color_mode='rgb',
            target_size=None,
            interpolation='nearest',
        )
        # convert it to an array
        img_array = np.array(img)
        # append the array to X
        X.append(img_array)
        # append the numeric target to y
        base = 0
        if i != 0:
            base = 1
        y.append(base)
        print(base)


X = np.array(X)
y = np.array(y)

# shuffle the data
shuffler = np.random.permutation(len(X))
X = X[shuffler]
y = y[shuffler]

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [37]:
X.shape

(377, 650, 650, 3)

## 4. Feature extraction + Logistic Regression

In [38]:
out_features_vector = base_model.predict(X)
out_features_vector.shape

(377, 20, 20, 512)

In [39]:
X.shape

(377, 650, 650, 3)

In [40]:
# To train the Logistic Regression model, we need to flatten out_features_vector
X_reshaped = out_features_vector.reshape((377,20*20*512))
X_reshaped.shape

(377, 204800)

In [41]:
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [42]:
# Train Test split
Xtrain,Xtest,ytrain,ytest = train_test_split(X_reshaped,
                                             y,
                                             test_size=.30,
                                             random_state=42)

In [43]:
# Train a Logistic Regression model
log_reg = LogisticRegression()
log_reg = log_reg.fit(Xtrain,ytrain)

In [44]:
ypred_train= log_reg.predict(Xtrain)
ypred_test = log_reg.predict(Xtest)

In [45]:
# Determine train and test scores
print(f"accuracy score on train: {accuracy_score(ytrain,ypred_train)}")
print(f"accuracy score on test: {accuracy_score(ytest,ypred_test)}")

accuracy score on train: 1.0
accuracy score on test: 1.0


In [46]:
# Save the model
with open('./data/log_reg.bin', 'wb') as f:
    pickle.dump(log_reg, f)

In [48]:
tf.keras.models.save_model(base_model, "./data/logreg_base_model")

INFO:tensorflow:Assets written to: ./data/logreg_base_model/assets
